In [ ]:
from genscai.tools import MCPClient

import nest_asyncio
nest_asyncio.apply()

mcp_client = MCPClient()

tools = mcp_client.list_tools()
for tool in tools:
    print(f"Tool Name: {tool.name}")    
    print(f"Description: {tool.description}")
    print(f"Input schema: {tool.inputSchema}")
    print("-" * 40)


name='search_research_articles' description='\n    Search for current research articles in infectious diseases and disease modeling per a given search request.\n\n    Args:\n        search_request: The information that the user is looking for.\n\n    Returns:\n        Current research articles on infectious diseases and disease modeling for the given topic.\n    ' inputSchema={'properties': {'search_request': {'title': 'Search Request', 'type': 'string'}}, 'required': ['search_request'], 'title': 'search_research_articlesArguments', 'type': 'object'} annotations=None
Tool Name: search_research_articles
Description: 
    Search for current research articles in infectious diseases and disease modeling per a given search request.

    Args:
        search_request: The information that the user is looking for.

    Returns:
        Current research articles on infectious diseases and disease modeling for the given topic.
    
Input schema: {'properties': {'search_request': {'title': 'Searc

AttributeError: 'Tool' object has no attribute 'outputSchema'